This notebooks help to prepare dataset for training flow matching model.

We convert images to VAE latents, compute CLIP embeddings and saves them locally for training.

In [3]:
#folder with images dataset
images_folder='/home/vlad/Documents/TreesDataset'

# where to save VAE latents
save_latents_dir = './latents_tree'

# image size for VAE
image_size=256

# generate also horizontal flip version for each image
add_flip = True

# apply grayscale to saved latents
vae_latents_grayscale = False

In [4]:
import os
import torchvision.transforms as T
from kemsekov_torch.utils import PadToMultiple
import tqdm
from vae import VaeEncoder
from clip_emb import *

device='cuda'
clip = CLIPEmbedder(device=device)
vae = VaeEncoder(device=device)

tr = T.Compose([
    T.ToTensor(),
    T.Resize(image_size,T.InterpolationMode.NEAREST),
    T.RandomHorizontalFlip(),
])
flip = T.RandomHorizontalFlip()

grayscale = T.Grayscale() if vae_latents_grayscale else lambda x:x 

pad = PadToMultiple(64)

image_formats=['.png','.jpg','.jpeg','.webp']
def save(clip, device, pad, save_folder, i, f, im):
    latents = vae.encode(pad(im)).half()
    emb = clip.image_to_embedding(f).half()
    torch.save(
            {
                "im_path":f,
                'vae_latents':latents,
                'clip_emb':emb,
            },
            os.path.join(save_folder,str(i)+".pt")
        )

for dir_path,subf,files in os.walk(images_folder):
    files = [os.path.join(dir_path,f) for f in files if [f.endswith(v) for v in image_formats]]
    class_ = os.path.split(dir_path)[-1]
    if len(files)==0: continue
    save_folder = os.path.join(save_latents_dir,class_)
    os.makedirs(save_folder,exist_ok=True)
    files_r = files
    data = tqdm.tqdm(enumerate(files_r),total=len(files_r),desc=class_)
    for i,f in data:
        im=tr(Image.open(f).convert("RGB"))
        im = grayscale(im)
        save(clip, device, pad, save_folder, i, f, im)
        if add_flip:
            save(clip, device, pad, save_folder, str(i)+"_flip", f, flip(im))

/home/vlad/Programs/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
a1: 100%|██████████| 10/10 [00:00<00:00, 10.10it/s]
